In [2]:
from pathlib import Path
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

In [3]:
from open_pref_eval.evaluation import evaluate_model
from open_pref_eval.plot.radar import radar_plot
from open_pref_eval.datasets.genies import dist2datasets, GENIES
from open_pref_eval.datasets.ethics import get_ethics_datasets
from open_pref_eval.datasets import load_dataset_n
from open_pref_eval.datasets import ds2name
from open_pref_eval.plot.radar import radar_plot

In [17]:
import pandas as pd
from reprpo.models.load import load_model, print_trainable_parameters

In [18]:
f = Path('/workspace/repr-preference-optimization/outputs/princeton-nlp_Llama-3-Base-8B-SFT_HRAKL_us_history_textbook/2024-09-18_10-34-31-adapter/HRAKL-us_history_textbook')
sorted(f.glob('**/*'))

# TODO turn into a df we can filter

[PosixPath('/workspace/repr-preference-optimization/outputs/princeton-nlp_Llama-3-Base-8B-SFT_HRAKL_us_history_textbook/2024-09-18_10-34-31-adapter/HRAKL-us_history_textbook/adapter_config.json'),
 PosixPath('/workspace/repr-preference-optimization/outputs/princeton-nlp_Llama-3-Base-8B-SFT_HRAKL_us_history_textbook/2024-09-18_10-34-31-adapter/HRAKL-us_history_textbook/adapter_model.safetensors')]

In [25]:
out_dir = Path('/workspace/repr-preference-optimization/outputs/')
f = out_dir.glob('princeton-nlp_Llama-3-Base-8B-SFT*/**/adapter_config.json')
def path2dict(p):
    assert p.is_dir()
    d1 = p.name # adapter - ds
    d2 = p.parent.name # date
    d3 = p.parent.parent.name # base method ds
    adapter, dsname = str(d1).split('-', 1)
    time = str(d2).replace('-adapter', '')
    # print(d3, p)
    base_method, adapter2, dsname2 = str(d3).split('_', 2)
    return dict(adapter= adapter,
                adapter2 = adapter2,
                dsname = dsname,
                dsname2 = dsname2,
                base_method = base_method,
                time = time,
                path = p)



df_paths = pd.DataFrame([path2dict(p.parent) for p in f])
df_paths

,adapter,adapter2,dsname,dsname2,base_method,time,path
0,DPO,Llama-3-Base-8B-SFT,alpaca_mmlu,DPO_alpaca_mmlu,princeton-nlp,2024-09-16_01-10-53,/workspace/repr-preference-optimization/output...
1,DPO,Llama-3-Base-8B-SFT,alpaca_mmlu,DPO_alpaca_mmlu,princeton-nlp,2024-09-16_07-29-01,/workspace/repr-preference-optimization/output...
2,DPO,Llama-3-Base-8B-SFT,alpaca_mmlu,DPO_alpaca_mmlu,princeton-nlp,2024-09-16_09-45-40,/workspace/repr-preference-optimization/output...
3,DPO,Llama-3-Base-8B-SFT,alpaca_mmlu,DPO_alpaca_mmlu,princeton-nlp,2024-09-16_14-32-31,/workspace/repr-preference-optimization/output...
4,DPO,Llama-3-Base-8B-SFT,alpaca_mmlu,DPO_alpaca_mmlu,princeton-nlp,2024-09-16_17-46-51,/workspace/repr-preference-optimization/output...
...,...,...,...,...,...,...,...
131,ETHER,Llama-3-Base-8B-SFT,alpaca_short,ETHER_alpaca_short,princeton-nlp,2024-09-17_04-28-48,/workspace/repr-preference-optimization/output...
132,HRAKL,Llama-3-Base-8B-SFT,us_history_textbook,HRAKL_us_history_textbook,princeton-nlp,2024-09-18_10-34-31,/workspace/repr-preference-optimization/output...
133,HRAKL,Llama-3-Base-8B-SFT,us_history_textbook,HRAKL_us_history_textbook,princeton-nlp,2024-09-18_12-27-36,/workspace/repr-preference-optimization/output...
134,HRAKL,Llama-3-Base-8B-SFT,us_history_textbook,HRAKL_us_history_textbook,princeton-nlp,2024-09-18_21-42-39,/workspace/repr-preference-optimization/output...


In [20]:
fs = [
    Path('/workspace/repr-preference-optimization/outputs/princeton-nlp_Llama-3-Base-8B-SFT_HRAKL_us_history_textbook/2024-09-18_10-34-31-adapter/HRAKL-us_history_textbook'),
    Path('/workspace/repr-preference-optimization/outputs/princeton-nlp_Llama-3-Base-8B-SFT_DPO_us_history_textbook/2024-09-17_05-42-35-adapter/DPO-us_history_textbook/'),
    Path('/workspace/repr-preference-optimization/outputs/princeton-nlp_Llama-3-Base-8B-SFT_DPO_math/2024-09-17_00-24-39-adapter/DPO-math/'),
    Path('/workspace/repr-preference-optimization/outputs/princeton-nlp_Llama-3-Base-8B-SFT_ETHER_us_history_textbook/2024-09-17_05-23-32-adapter/ETHER-us_history_textbook/'),
    Path('/workspace/repr-preference-optimization/outputs/princeton-nlp_Llama-3-Base-8B-SFT_Sideout_us_history_textbook/2024-09-15_12-23-37-adapter/Sideout-us_history_textbook/')
 ]

In [ ]:
# !tree /workspace/repr-preference-optimization/outputs/princeton-nlp_Llama-3-Base-8B-SFT_HRAKL_us_history_textbook/2024-09-18_10-34-31-adapter

In [40]:
from reprpo.train.pl_base import TrainingArguments
args = TrainingArguments(verbose=True)

In [27]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
from peft import AutoPeftModelForCausalLM, get_peft_model, PeftConfig, PeftModelForCausalLM
model_name = fs[0]
peft_config = PeftConfig.from_pretrained(model_name)
base_model = AutoModelForCausalLM.from_pretrained(model_name)
model = PeftModelForCausalLM.from_pretrained(
    base_model,
    model_name, config=peft_config, adapter_name=f.stem)
tokenizer = AutoTokenizer.from_pretrained(args.base_model)
model

In [44]:
for f in fs[1:]:
    model.load_adapter(f, f.stem)


In [ ]:
model.peft_config

In [ ]:

model

In [46]:
model = model.cuda()

In [29]:

# ## Eval
# eval on ethics, GENIES, and our train dataset
N = None
N = 64
dataset = 'us_history_textbook'
datasets = [
    load_dataset_n('wassname/genie_dpo', name=args.dataset, split='train', N=N),
    load_dataset_n('wassname/genie_dpo', name=args.dataset, split='test', N=N),
]
datasets += dist2datasets(GENIES, N=N, source=[args.dataset]) # our hard OOS test
datasets += get_ethics_datasets(N=N)
datasets += [load_dataset_n('wassname/genie_dpo', name=name, split='test', N=N) for name in ['math_make_questions', 'truthful_qa', 'wrong_arc', 'ranking_logic',
 'math', 'sycophancy_mimicry'
]]

In [ ]:
datasets

In [ ]:
res, df_res2 = evaluate_model(model=model, 
                                tokenizer=tokenizer, 
                                datasets=datasets,
                                    batch_size=args.batch_size*2,
                                    bf16=True,
                                    torch_empty_cache_steps=100, verbose=args.verbose)

In [ ]:
res['correct'].unstack().T

In [50]:
import gc, torch
base_model = tokenizer = model = None
gc.collect()
torch.cuda.empty_cache()

In [51]:
import pandas as pd

ds_name_train = args.dataset.replace("genie_dpo-", "")
model_name = args.base_model.split("/")[-1]
adapter_name = type(args).__name__
finetune_name = f"{adapter_name}-{ds_name_train}"

In [52]:

def key_metrics(df_res2):
    ds_name_train = ds2name(datasets[0])
    ds_name_test = ds2name(datasets[1])
    ds_name_oos = ds2name(datasets[2])
    ds_name_rnd = ds2name(datasets[3])

    df_res = (
        df_res2.groupby(["dataset", "adapter"], dropna=False)["correct"]
        .mean()
        .unstack()
        .T
    )
    rel_acc = df_res.loc[finetune_name] / df_res.loc["base"]

    # metric: do we retrain train coherency?
    df_res_logp = (
        df_res2.groupby(["dataset", "adapter"], dropna=False)["_chosen_logps"]
        .mean()
        .unstack()
        .T
    )
    rel_coherency = df_res_logp.loc[finetune_name] - df_res_logp.loc["base"]

    # metric: do we retrain train coherency?
    c = df_res_logp = (
        df_res2.groupby(["dataset", "adapter"], dropna=False)["_chosen_logps"]
        .mean()
        .unstack()
        .T.loc[finetune_name]
    )
    r = df_res_logp = (
        df_res2.groupby(["dataset", "adapter"], dropna=False)["_rejected_logps"]
        .mean()
        .unstack()
        .T.loc[finetune_name]
    )
    cho_rej_coh = c - r

    def fmt(s):
        return s.replace("genie_dpo-", "")

    # TODO make multiple cols of index

    df_metrics = pd.DataFrame(
        [
            # accuracy increase over base measured generalisaton on increasing distribution shifts
            ["acc[pi/base]", "train", fmt(ds_name_train), rel_acc[ds_name_train]],
            ["acc[pi/base]", "test", fmt(ds_name_test), rel_acc[ds_name_test]],
            ["acc[pi/base]", "oos", fmt(ds_name_oos), rel_acc[ds_name_oos]],
            [
                "acc[pi/base]",
                "rnd",
                fmt(ds_name_rnd),
                rel_acc[ds_name_rnd],
            ],  # probobly wont go up as it's unrelated
            # we want to see if it retains coherency vs the base on chosen answers
            [
                "coherency[pi-base]",
                "train",
                fmt(ds_name_train),
                rel_coherency[ds_name_train],
            ],
            [
                "coherency[pi-base]",
                "test",
                fmt(ds_name_test),
                rel_coherency[ds_name_test],
            ],
            [
                "coherency[pi-base]",
                "oos",
                fmt(ds_name_oos),
                rel_coherency[ds_name_oos],
            ],
            [
                "coherency[pi-base]",
                "rnd",
                fmt(ds_name_rnd),
                rel_coherency[ds_name_rnd],
            ],
            # we want to see if it retains chosen vs rejected
            [
                "coherency[cho-rej]",
                "train",
                fmt(ds_name_train),
                cho_rej_coh[ds_name_train],
            ],
            [
                "coherency[cho-rej]",
                "test",
                fmt(ds_name_test),
                cho_rej_coh[ds_name_test],
            ],
            [
                "coherency[cho-rej]",
                "oos",
                fmt(ds_name_oos),
                cho_rej_coh[ds_name_oos],
            ],
            [
                "coherency[cho-rej]",
                "rnd",
                fmt(ds_name_rnd),
                cho_rej_coh[ds_name_rnd],
            ],
        ],
        columns=["metric", "split", "dataset", "value"],
    )[["metric", "split", "value", "dataset"]]
    df_metrics = df_metrics.set_index(["metric", "split"])
    df_metrics = df_metrics["value"].unstack()
    df_metrics.index.name = f"{adapter_name}\dist shift"

    return df_metrics

In [ ]:
df_res2

In [ ]:
df_res = (
    df_res2.groupby(["dataset", "adapter"], dropna=False)["correct"]
    .mean()
    .unstack()
    .T
)
df_res.columns = [d.replace("genie_dpo-", "") for d in df_res.columns]


df_metrics = key_metrics(df_res2)


from collections import OrderedDict

ds_alias = OrderedDict(
    list(zip(["train", "test", "oos", "rnd"], [ds2name(d) for d in datasets]))
)


print()
print(df_metrics.round(3).to_markdown())
print("""Table 1: Key metrics (adapter over base model)\n""")

cols = [v.replace("genie_dpo-", "") for v in ds_alias.values()]
df_res2 = df_res[cols]
df_res2.columns = list(ds_alias.keys())
df_res2.index.name = "adapter/ds"
print(df_res2.round(3).to_markdown())
print("""Table 2: Absolute accuracy\n""")

df_final = df_metrics.loc["acc[pi/base]"].to_frame(adapter_name).T
df_final = df_final * 100 - 100  # percentage points
df_final.index.name = "acc_inc/eval_ds [pp]"
print(df_final.round(3).to_markdown())
print(
    f"""Table 3: Accuracy increase (in percentage points) after training with named adapter on `{args.dataset}` compared to base model `{args.base_model}` for various distribution shifts:"""
)
for k, v in ds_alias.items():
    print(f"- `{k}`: `{v}`")
print()

relacc = df_final.iloc[0, :]
eps = 1e-6
relrelacc = ((relacc + eps) / (relacc["train"] + eps)).drop("train")
df_relrel = relrelacc.to_frame(f"{adapter_name}").T
df_relrel.index.name = "acc_inc/acc_inc_train"
print(df_relrel.round(3).to_markdown())
print(
    f"""Table 4: Percent accuracy increase (over base) compared to that of the training dataset `{ds_alias['train']}` [in percentage points]. It measures what fraction of the learning from train generalised to other splits\n"""
)